In [106]:
import pandas as pd
import scipy as sp
import numpy as np
import sklearn as sk
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import random
import torch
import math
import gc
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import plotly.graph_objects as go
import plotly.express as px


from abc import ABC, abstractmethod

In [2]:
!unzip archive_2.zip

unzip:  cannot find or open archive_2.zip, archive_2.zip.zip or archive_2.zip.ZIP.


In [3]:
train_df = pd.read_csv('mnist_train.csv', sep=',')

In [4]:
test_df = pd.read_csv('mnist_test.csv', sep=',')

In [5]:
train_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = train_df.drop('label', axis=1)
y = train_df['label']

In [8]:
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
X_test = scaler.transform(X_test)

## Pytorch implementation

In [11]:
class EqualityFunction(torch.autograd.Function):

  def __call__(self, x):
    return x


In [12]:
class LayerBase(ABC):
  def __init__(self, activation):
    self._activation = activation

  @abstractmethod
  def forward(self, X):
    pass
  
class HiddenEulerLayer(LayerBase):
  def __init__(self, n_neurons, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons, n_neurons)
    self._normalizator = torch.nn.BatchNorm1d(n_neurons, track_running_stats=False, affine=False)
    # TODO: Тут неплохо инициализировать в зависимости от activation
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    self._h = h

  def forward(self, X):
    X = self._normalizator(X)
    return X + self._h * self._activation(X @ self.W)

class HiddenEulerLayerTest(HiddenEulerLayer):
  def forward(self, X):
    return X + self._h * self._activation(X @ self.W)

class HiddenRungeLayer(LayerBase):
  def __init__(self, n_neurons, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons, n_neurons)
    self._normalizator = torch.nn.BatchNorm1d(n_neurons, track_running_stats=False, affine=False)
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    self._h = h

  def forward(self, X):
    X = self._normalizator(X)
    k1 = self._activation(X @ self.W)
    k2 = self._activation((X + self._h / 2 * k1) @ self.W)
    k3 = self._activation((X + self._h / 2 * k2) @ self.W)
    k4 = self._activation((X + self._h * k3) @ self.W)
    
    return X + self._h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

class ConvolutionalLayer(LayerBase):
  def __init__(self, n_neurons_in, n_neurons_out, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons_in, n_neurons_out)
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    self._h = h

  def forward(self, X):
    return self._h * self._activation(X @ self.W)

In [13]:
class DeepCNNBase(ABC):

  ACTIVASIONS = {
    'equality': EqualityFunction(),
    'relu': torch.nn.ReLU(),
    'sigmoid': torch.sigmoid,
    'hyperbolic_tg': torch.tanh,
    'softmax': torch.nn.LogSoftmax(dim=1),
    'leakyReLU': torch.nn.LeakyReLU(0.1)
  }

  def __init__(self, n_layers=50, activation='equality'):
    self._n_layers = n_layers
    if activation not in self.ACTIVASIONS:
      raise RuntimeError('''Invalid activation function name. 
      Use one of: {}'''.format(','.join(self.ACTIVASIONS.keys())))
    self._activation = self.ACTIVASIONS[activation]
    self._layers = []

  @abstractmethod
  def fit(self):
    pass
  
  @abstractmethod
  def predict(self):
    pass


In [14]:
class DeepCNNClassifier(DeepCNNBase):

  METRIC_FUNCTIONS = {
    'accuracy': metrics.accuracy_score,
    'f1': metrics.f1_score,
    'precision': metrics.precision_score,
    'recall': metrics.recall_score,
  }

  OPTIMIZERS = {
      'rmsprop': torch.optim.RMSprop,
      'adam': torch.optim.Adam,
      'adamax': torch.optim.Adamax,
      'adamw': torch.optim.AdamW,
      'adadelta': torch.optim.Adadelta,
  }

  METHODS = {
      'euler': HiddenEulerLayer,
      'runge': HiddenRungeLayer
  }

  def __init__(self, n_layers=3, n_features=None, activation='equality', metric='accuracy', 
               max_iter=10, tol=1e-4, batch_size=-1, max_opt_iter=100,
               opt_tol=1e-5, optimizer='adam', lr=0.1, l2=1e-3, h=1,
               method='euler', verbose=False):
    super().__init__(n_layers, activation)
    if metric not in self.METRIC_FUNCTIONS:
      raise RuntimeError('''Invalid metric function name. 
      Use one of: {}'''.format(','.join(self.METRIC_FUNCTIONS.keys())))
    
    if optimizer not in self.OPTIMIZERS:
      raise RuntimeError('''Invalid optimizer name. 
      Use one of: {}'''.format(','.join(self.OPTIMIZERS.keys())))
    
    if method not in self.METHODS:
      raise RuntimeError('''Invalid discretization method name. 
      Use one of: {}'''.format(','.join(self.METHODS.keys())))

    self._metric = self.METRIC_FUNCTIONS[metric]
    self._optimizer = self.OPTIMIZERS[optimizer]
    self._layerclass = self.METHODS[method]

    self._max_iter = max_iter
    self._tol = tol
    self._verbose = verbose
    self._batch_size = batch_size
    self._max_opt_iter = max_opt_iter
    self._opt_tol = opt_tol
    self._lr = lr
    self._l2 = l2
    self._n_features = n_features
    self._h = h

    self.metrics = []
    self.errors = []

  def fit(self, X, y, X_test=None, y_test=None):
    X = torch.Tensor(X)
    if X_test is not None:
      X_test = torch.Tensor(X_test)
    if y_test is not None:
      y_test = torch.Tensor(y_test).type(torch.LongTensor)

    classes_count = np.unique(y).shape[0]
    y = torch.Tensor(y.reshape((y.shape[0]))).type(torch.LongTensor)

    self._layers = []
    input_features_count = X.shape[1] 
    features_count = self._n_features or X.shape[1]
    # Создаем слои нейронной сети
    self._layers.append(ConvolutionalLayer(input_features_count, features_count, self.ACTIVASIONS['equality'], h=self._h))
    for _ in range(self._n_layers):
      self._layers.append(self._layerclass(features_count, self._activation, h=self._h))
    # Выходной слой
    self._layers.append(ConvolutionalLayer(features_count, classes_count, self.ACTIVASIONS['softmax'], h=self._h))
    # Обучение
    iter_count = 0
    prev_metric = None
    while iter_count < self._max_iter:
      lr = self.__get_lr(iter_count)
      for stop_layer_idx in range(len(self._layers) - 1, -1, -1):
        # Оптимизируем
        self.__optimize(stop_layer_idx, X, y, lr)
      X_copy = self.__forward(X.clone())
      y_pred = self.__out_to_labels(X_copy)
      metric = self._metric(y, y_pred)
      loss = torch.nn.NLLLoss()
      error = loss(X_copy, y)
      if X_test is not None and y_test is not None:
        X_copy = self.__forward(X_test.clone())
        y_pred = self.__out_to_labels(X_copy)
        metric_test = self._metric(y_test, y_pred)
        error_test = loss(X_copy, y_test)
        self.metrics.append((metric, metric_test))
        self.errors.append((error.item(), error_test.item()))
      else:
        self.metrics.append(metric)
        self.errors.append(error.item())
      if self._verbose:
        print('=' * 10)
        print('Temp iteration: {}'.format(iter_count + 1))
        print('Metric: {}'.format(metric))
        print('Error: {}'.format(error))
        if X_test is not None and y_test is not None:
          print('Test metric: {}'.format(metric_test))
          print('Test error: {}'.format(error_test))
        print('=' * 10 + '\n')
      
      if prev_metric is not None and np.abs(metric - prev_metric) < self._tol:
        break 
      prev_metric = metric
      iter_count += 1
  
  def predict(self, X):
    X_copy = torch.Tensor(X)
    X_copy = self.__forward(X_copy)
    return self.__out_to_labels(X_copy)
  
  def score(self, X, y):
    y_pred = self.predict(X)
    return self._metric(y, y_pred)

  def __forward(self, X, from_i=None, to_i=None):
    if from_i is None:
      from_i = 0
    if to_i is None:
      to_i = len(self._layers)
    for i in range(from_i, to_i):
      X = self._layers[i].forward(X)
    return X

  def __get_lr(self, iter):
    drop = 0.5
    epochs_drop = 10.0
    return self._lr# * math.pow(drop, math.floor((1+iter)/epochs_drop))

  def __optimize(self, layer_idx, X, y, lr):
    iter_count = 0
    prev_error = None
    self._layers[layer_idx].W.requires_grad = True
    optimizer = self._optimizer([self._layers[layer_idx].W], lr=lr,
                                weight_decay=self._l2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.75)
    while iter_count < self._max_opt_iter:
      optimizer.zero_grad(set_to_none=True)
      # Градиент на текущем слое
      if self._batch_size >= X.shape[0] or self._batch_size == -1:
        indicies = np.arange(X.shape[0])
      else:
        indicies = random.sample_without_replacement(n_population=X.shape[0], 
                                                     n_samples=self._batch_size)
      X_copy = X[indicies].clone().detach()
      X_copy = self.__forward(X_copy)
      # Кросс-энтропия
      loss = torch.nn.NLLLoss()
      error = loss(X_copy, y[indicies])
      if prev_error is not None and np.abs(prev_error - error.item()) < self._opt_tol:
        break
      error.backward()

      optimizer.step()
      prev_error = error.item()
      
      iter_count += 1
    self._layers[layer_idx].W.requires_grad = False
    self._layers[layer_idx].W.grad = None
    if self._verbose:
      if layer_idx == 0:
        print('Error on input layer: ', end='')
      elif layer_idx == len(self._layers) - 1:
        print('Error on output layer: ', end='')
      else:
        print('Error on layer {}: '.format(layer_idx), end='')
      print('{} (iter_count: {})'.format(prev_error, iter_count))
  
  def plot_metric(self):
    train_metrics = [x[0] for x in self.metrics]
    test_metrics = [x[1] for x in self.metrics]
    x = [i for i in range(1, len(train_metrics)+1)]
    _, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=400)

    axes.plot(x, train_metrics, 'b-', label='train_metrics')
    axes.plot(x, test_metrics, 'r-', label='test_metrics')

    axes.xaxis.set_major_locator(MaxNLocator(integer=True))
    axes.set_xlabel('number of iteration')
    axes.set_ylabel('metric')
    axes.legend(loc="upper center")
 
    axes.grid()
    plt.show()
  
  def plot_errors(self):
    train_errors = [x[0] for x in self.errors]
    test_errors = [x[1] for x in self.errors]
    x = [i for i in range(1, len(train_errors)+1)]
    _, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=400)

    axes.plot(x, train_errors, 'b-', label='train_errors')
    axes.plot(x, test_errors, 'r-', label='test_errors')

    axes.xaxis.set_major_locator(MaxNLocator(integer=True))
    axes.set_xlabel('number of iteration')
    axes.set_ylabel('errors')
    axes.legend(loc="upper center")
 
    axes.grid()
    plt.show()

  def __out_to_labels(self, X_out):
    X_out = X_out.detach().numpy()
    return np.argmax(X_out, axis=1)


In [127]:
clf = DeepCNNClassifier(verbose=True,
                        n_layers=5,
                        n_features=800,
                        batch_size=600, 
                        max_opt_iter=20, 
                        max_iter=100,
                        activation='relu',
                        optimizer='adam',
                        tol=-1,
                        opt_tol=-1,
                        lr=1e-2, 
                        l2=1e-2,
                        method='euler')

In [195]:
clf = DeepCNNClassifier(verbose=True,
                        n_layers=3,
                        n_features=1200,
                        batch_size=600,
                        max_opt_iter=100, 
                        max_iter=100,
                        activation='leakyReLU',
                        optimizer='adam',
                        tol=-1,
                        opt_tol=-1,
                        lr=1e-3, 
                        l2=1e-3,
                        method='euler',
                        h=0.5)

In [ ]:
%%time
clf.fit(X, y.to_numpy(), X_test, y_test)


In [ ]:
clf.plot_metric()

In [ ]:
clf.plot_errors()

In [ ]:
clf.score(X, y)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=300).fit(X, y)


In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
metrics.accuracy_score(y_test, y_pred)

0.9214

#Тест

In [15]:
def test_func(x):
  return (x-1)**2 + 4


In [78]:
def test_rozenbrok(x):
  if len(x.shape) == 1:
    return (1-x[0])**2 + 100*(x[1]-x[0]**2)**2
  return (1-x[:, 0])**2 + 100*(x[:, 1]-x[:, 0]**2)**2

In [205]:
class TestCNN(DeepCNNBase):

  OPTIMIZERS = {
      'rmsprop': torch.optim.RMSprop,
      'adam': torch.optim.Adam,
      'adamax': torch.optim.Adamax,
      'adamw': torch.optim.AdamW,
      'adadelta': torch.optim.Adadelta,
  }

  METHODS = {
      'euler': HiddenEulerLayerTest,
      'runge': HiddenRungeLayer
  }

  def __init__(self, test_function, n_layers=3, n_features=None, activation='equality',
               max_iter=10, tol=1e-4, batch_size=-1, max_opt_iter=100,
               opt_tol=1e-5, optimizer='adam', lr=0.1, l2=1e-3, h=1,
               method='euler', verbose=False):
    super().__init__(n_layers, activation)
    
    if optimizer not in self.OPTIMIZERS:
      raise RuntimeError('''Invalid optimizer name. 
      Use one of: {}'''.format(','.join(self.OPTIMIZERS.keys())))
    
    if method not in self.METHODS:
      raise RuntimeError('''Invalid discretization method name. 
      Use one of: {}'''.format(','.join(self.METHODS.keys())))

    self._optimizer = self.OPTIMIZERS[optimizer]
    self._layerclass = self.METHODS[method]

    self._max_iter = max_iter
    self._tol = tol
    self._verbose = verbose
    self._max_opt_iter = max_opt_iter
    self._opt_tol = opt_tol
    self._lr = lr
    self._l2 = l2
    self._n_features = n_features
    self._h = h

    self.errors = []
    self._layers = []

    self._test_func = test_function 

  def fit(self, X):
    X = torch.Tensor(X)

    input_features_count = len(X)
    # Создаем слои нейронной сети
    for _ in range(self._n_layers):
      self._layers.append(self._layerclass(input_features_count, self._activation, h=self._h))
    # Обучение
    iter_count = 0
    loss_array = []
    x_array = []
    while iter_count < self._max_iter:
      lr = self.__get_lr(iter_count)
      for stop_layer_idx in range(len(self._layers) - 1, -1, -1):
        # Оптимизируем
        self.__optimize(stop_layer_idx, X, lr)
      X_copy = self.__forward(X.clone())

      loss = self._test_func(X_copy)
      if self._verbose:
        print('=' * 10)
        print('Temp iteration: {}'.format(iter_count + 1))
        print('Error: {}'.format(loss))
        print('=' * 10 + '\n')
      loss_array.append(loss.item())
      if X_copy.shape[0] > 1:
        x_array.append(X_copy.numpy())
      else:
        x_array.append(X_copy.item())
      iter_count += 1
    print('received minimum:', loss.item())
    if X.shape[0] > 1:
      self.draw_on_epoch2d(x_array, loss_array)
    else:
      self.draw_on_epoch(x_array, loss_array)

  def predict(self):
    pass
  
  def __forward(self, X, from_i=None, to_i=None):
    if from_i is None:
      from_i = 0
    if to_i is None:
      to_i = len(self._layers)
    for i in range(from_i, to_i):
      X = self._layers[i].forward(X)
    return X

  def __get_lr(self, iter):
    drop = 0.5
    epochs_drop = 10.0
    return self._lr# * math.pow(drop, math.floor((1+iter)/epochs_drop))

  def __optimize(self, layer_idx, X, lr):
    iter_count = 0
    prev_error = None
    self._layers[layer_idx].W.requires_grad = True
    optimizer = self._optimizer([self._layers[layer_idx].W], lr=lr,
                                weight_decay=self._l2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.75)
    while iter_count < self._max_opt_iter:
      optimizer.zero_grad(set_to_none=True)
      # Градиент на текущем слое
      X_copy = X.clone().detach()
      X_copy = self.__forward(X_copy)
      # Кросс-энтропия
      loss = self._test_func(X_copy)
      loss.backward()

      optimizer.step()
      prev_error = loss.item()
      
      iter_count += 1
    self._layers[layer_idx].W.requires_grad = False
    self._layers[layer_idx].W.grad = None
    if self._verbose:
      print('Error on layer {}: '.format(layer_idx), end='')
      print('{} (iter_count: {})'.format(prev_error, iter_count))

  def draw_on_epoch(self, x_ar, y_ar):
    _, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=400)
    from_x = min(x_ar) - 0.1 * (max(x_ar) - min(x_ar))
    to_x = max(x_ar) + 0.1 * (max(x_ar) - min(x_ar))
    X = np.arange(from_x, to_x, 0.01)
    myhexline = '#76a5ba'
    myhexarrow = '#ba567f'
    axes.plot(X, self._test_func(X), color=myhexline, label='$(x-1)^2+4$', zorder=1)
    for i in range(len(x_ar)-1):
      arrow_lenght = np.sqrt((x_ar[i+1]-x_ar[i])**2 +(y_ar[i+1]-y_ar[i])**2)
      axes.arrow(x_ar[i], y_ar[i], x_ar[i+1]-x_ar[i], y_ar[i+1]-y_ar[i], length_includes_head=True,\
                width=0.01, color = myhexarrow,head_width=0.1, head_length=0.2*arrow_lenght,shape='full', zorder=2)
    
    axes.set_xlabel('x')
    axes.set_ylabel('y')
    axes.legend(loc="upper center")
    
    axes.grid(zorder=0)
    plt.show()

  def draw_on_epoch2d(self, x_ar, y_ar):
    x1_ar = list(map(lambda ar: ar[0], x_ar))
    x2_ar = list(map(lambda ar: ar[1], x_ar))

    from_x = min(x1_ar) - 0.5 * (max(x1_ar) - min(x1_ar))
    from_x = from_x if from_x < -2 else -2
    to_x = max(x1_ar) + 0.5 * (max(x1_ar) - min(x1_ar))
    to_x = to_x if to_x > 2 else 2

    from_y = min(x2_ar) - 0.5 * (max(x2_ar) - min(x2_ar))
    from_y = from_y if from_y < -1 else -1
    to_y = max(x2_ar) + 0.5 * (max(x2_ar) - min(x2_ar))
    to_y = to_y if to_y > 3 else 3
    x = np.linspace(from_x, to_x, 200)
    y = np.linspace(from_y, to_y, 200)

    x_result = []
    for x_coord in x:
      x_result += [x_coord] * y.shape[0]

    y_result = []
    for i in range(x.shape[0]):
      y_result += list(y)

    surface_dots = np.array(list(zip(x_result, y_result)))
    
    surface_z = self._test_func(surface_dots)
    surface_z = surface_z.reshape((x.shape[0], y.shape[0]), order='F')
    fig = go.Figure(data=[go.Surface(z=surface_z, x=x, y=y)])

    fig.add_trace(go.Scatter3d(x=x1_ar, y=x2_ar, z=y_ar,
                              marker=dict(
                                  size=4,
                                  color='red',
                                  colorscale='Viridis',
                              ),
                              line=dict(
                                  color='red',
                                  width=4
                                  )
                              )
    )
    fig.update_layout(title='Rozenbrock function', autosize=True,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()

    # Contour
    fig2 = go.Figure(data=[go.Contour(z=surface_z, x=x, y=y)])

    fig2.add_trace(go.Scatter(x=x1_ar, y=x2_ar,
                              marker=dict(
                                  size=4,
                                  color='red',
                                  colorscale='Viridis',
                              ),
                              line=dict(
                                  color='red',
                                  width=2
                                  )
                              )
    )
    fig2.update_layout(title='Rozenbrock function', autosize=True,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig2.show()
  
  

In [212]:
creg = TestCNN(verbose=True,
              test_function = test_rozenbrok,
              n_layers=6, 
              max_opt_iter=10, 
              max_iter=20,
              optimizer='adam',
              tol=-1,
              opt_tol=-1,
              lr=1e-2, 
              l2=1e-2,
              method='euler')

In [213]:
creg.fit([4, 3])

Error on layer 5: 4001.59130859375 (iter_count: 10)
Error on layer 4: 450.908447265625 (iter_count: 10)
Error on layer 3: 89.18363952636719 (iter_count: 10)
Error on layer 2: 31.561908721923828 (iter_count: 10)
Error on layer 1: 31.728830337524414 (iter_count: 10)
Error on layer 0: 81.27550506591797 (iter_count: 10)
Temp iteration: 1
Error: 61.135398864746094

Error on layer 5: 69.38611602783203 (iter_count: 10)
Error on layer 4: 235.04220581054688 (iter_count: 10)
Error on layer 3: 43.892784118652344 (iter_count: 10)
Error on layer 2: 31.191503524780273 (iter_count: 10)
Error on layer 1: 30.002161026000977 (iter_count: 10)
Error on layer 0: 78.52677917480469 (iter_count: 10)
Temp iteration: 2
Error: 58.646240234375

Error on layer 5: 62.41258239746094 (iter_count: 10)
Error on layer 4: 216.49581909179688 (iter_count: 10)
Error on layer 3: 43.365516662597656 (iter_count: 10)
Error on layer 2: 29.803590774536133 (iter_count: 10)
Error on layer 1: 28.91925048828125 (iter_count: 10)
Error